# Imports

In [42]:
import numpy as np
import time 
import re

import torch
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel

import openai
import tiktoken

In [2]:
data = np.load('datasets/emoji_apple_style.npy', allow_pickle=True).item()

In [3]:
data.keys()

dict_keys(['images', 'labels', 'embeddings', 'color_palette'])

In [9]:
data = list(data)

In [10]:
# sentence embedding model
# https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1

# limit of 512 word pieces, trained on length of 250 word pieces and might not work for longer texts

uri_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
tokenizer = AutoTokenizer.from_pretrained(uri_name)
model = AutoModel.from_pretrained(uri_name)

/Users/henrymei/anaconda3/envs/five-dollar-model/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Feature Generation

## Create embeddings

In [46]:
# Mean Pooling - Take average of all tokens
# see: https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1#pytorch-usage-huggingface-transformers
def _mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [47]:
def _tokenize(texts, tokenizer, max_length):
    """
    Generate token for input text. 
    :param texts: a list of input sentences or texts to be processed
    :param tokenizer: a Hugging Face tokenizer instance
    :param max_length: an optional parameter for padding/truncation of text strings
    :return: encoded inputs 
    """
    
    padding = True if max_length == 0 else 'max_length'
    
    # __call__ the tokenizer 
    return tokenizer(
        texts, 
        padding=padding,
        truncation=True,
        max_length=max_length,  # if left unset, uses model default 
        return_tensors='pt'  # return as torch tensors 
    )    

In [48]:
def _embed(encoded_input, model):
    """
    Take tokenized values and generate embeddings
    :param encoded_input: encoded inputs generated by tokenizer 
    :param model: a Hugging Face model instance
    :return: embedding vector
    """
    
    # Compute token embeddings
    with torch.no_grad():  # only need forward pass here
        embeddings_words = model(**encoded_input, return_dict=True)

    # Perform mean pooling
    # The attention mask ensures that padding tokens do not contribute to the averaged embedding.
    # purpose is to take variable length sequences and output fixed length ones 
    embeddings_sentence = _mean_pooling(embeddings_words, encoded_input['attention_mask'])

    # Normalize embeddings -- L2 = 1
    embeddings_sentence = F.normalize(embeddings_sentence, p=2, dim=1)
    
    return embeddings_sentence, embeddings_words

In [49]:
def get_sent_word_embeddings(labels, model, tokenizer, max_length=None):
    """
    Generate sentence embedding for input texts
    :param model: Hugging Face model instance
    :param tokenizer: Hugging Face tokenizer instance
    :param labels: input labels from our training data 
    :param max_length: maximum length for padding/truncation fo input stirngs
    :return: both mean-pooled sentence embedding and masked word embeddings 
    """
    encoded = _tokenize(labels, tokenizer, max_length)
    embeddings_sentences, embeddings_words = _embed(encoded, model)

    embeddings_words = embeddings_words['last_hidden_state'].detach().cpu().numpy()
    embeddings_sentences = embeddings_sentences.detach().cpu().numpy()

    return embeddings_sentences, embeddings_words

## GPT Augmentation

In [11]:
openai.organization = ''
openai.api_key='' # enter open ai API key here :)

In [33]:
def _compute_tokens(payload, encoding):
    """
    Estimate the number of tokens required in the request. 
    See: https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken
    
    :param payload: an array of dicts or an array of strings 
    :param encoding: a tiktoken encoder instance 
    :param model: the name of the openAI model used 
    :return: a count of tokens 
    """
    num_tokens = 0
    for message in payload:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        
        # if we pass a wellformed payload 
        if type(message) == dict:
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                
                # if there's a name, the role is not processed and so doesn't count to api usage
                if key == "name":  
                    num_tokens += -1 
        # if we pass a list of labels 
        else:
            num_tokens += len(encoding.encode(message))
    
    # pad the estimate with the structure of response; note: does not include actual response
    # every reply is primed with <im_start>assistant     
    num_tokens += 2  
    
    return num_tokens

In [34]:
def _create_payload(labels):
    prompt = """Take each string in the list provided, and write an alternate label for each one. These strings describe an image of a pixel video game map. These alternate labels should describe the same image as the original label, but use different words and a different sentence structure. Use simple or common words when writing the alternate labels. Assume you have the vocabulary of a 10 year old. Your output should have the same number of strings as the input list."""

    return [
        {"role": "system", "content": "You are a helpful assistant with excellent attention to detail. You only output python lists of strings according to the instructions you are given. Output the list on a single line, without any newlines. Make sure every list is closed properly"},
        {"role": "user", "content": f"{prompt} Here is the list of labels: {labels}"},            
    ]

In [38]:
def _call_gpt(messages, model):
    return openai.ChatCompletion.create(model=model, messages=messages)

In [39]:
# segment the data into sublists to not exceed api limits

# tiktoken is a fast open-source tokenizer by OpenAI.

# Given a text string (e.g., "tiktoken is great!") and an encoding (e.g., "cl100k_base"), a tokenizer can split the text string into a list of tokens (e.g., ["t", "ik", "token", " is", " great", "!"]).
# 
# Splitting text strings into tokens is useful because GPT models see text in the form of tokens. Knowing how many tokens are in a text string can tell you (a) whether the string is too long for a text model to process and (b) how much an OpenAI API call costs (as usage is priced by token).
def _chunk_labels(labels, encoding, threshold):
    """
    
    :param labels: feature inputs as a list of strings
    :param encoding: a tiktoken encoder instance
    :param threshold: a maximum token size to chunk the inputs into 
    :return: 
    """

    current_chunk, chunks = [], []
    count_tokens = 0
    label_tokens = _compute_tokens(labels, encoding)
    
    # step throw all labels 
    for label, tokens in zip(labels, label_tokens):
       
        # append labels to the current chunk and update our count
        current_chunk.append(label)
        count_tokens += tokens 
        
        # if the array exceeds the token threshold then.... 
        if count_tokens + 2 > threshold:
            
            # remove the last label 
            hold = current_chunk.pop()
            
            # complain if the label itself is so big that it's as big as the threshold
            if tokens > threshold:
                raise Exception(f"Element at position *{i}*, *'{[hold]}'* is too big: *{size} tokens* for this threshold: *{threshold} tokens*")
            
            # since we removed the offending label, the current chunk should be the right size
            chunks.append(current_chunk)
            
            # start a new arr with the one we popped out and reset our counter
            current_chunk = [hold] 
            count_tokens = 0
            
    # add the final set of labels 
    chunks.append(current_chunk)
    
    return chunks

In [44]:
def _process_result(result):
    """
    Filter out patterns that look like [' and '] 
    TODO: why don't we just look at those patterns directly instead of regex? 
    :param result: raw openAI API response 
    :return: processed answers 
    """
    answer = result.choices[0]['message']['content']                
    apostrophe_pattern = r"(?<=\w)'(?=[^,\]])|'(?=\w+?'\s)"
    answer = re.sub(apostrophe_pattern, '', answer)
    
    idx_open = answer.find("[")
    idx_close = answer.find(']') + 1 # +1 since indexing ignores current spot 
    
    return answer[idx_open:idx_close]

In [45]:
def get_gpt_alt_labels(labels, model='gpt-4o-mini', num_retries=3, debug=True):
    """
    Call GPT to generate alternate labels. 
    :param labels: list of human-annotated labels 
    :param num_retries: how many times do we try again? 
    :return: (a list of alternate labels, api call response status)
    """

    start = time.time()
    
    # it's probably clk100k_base (can pass to get_encoding()), but let's not assume
    encoding = tiktoken.encoding_for_model(model)
    prompt_size = _compute_tokens(_create_payload([]), encoding)
    
    # chunk the prompt to the right size; if we pass in everything, it's gonna timeout/fail
    threshold = 4000 - prompt_size 
    chunks = _chunk_labels(labels, encoding, threshold=threshold)

    
    if debug:
        print(f'Prompt size: {prompt_size}')
        print(f"split time = {time.time() - start}")
        print("Number of loops: ", len(chunks))
        
    labels, ans_arr = [], []
    

    for i, labels in enumerate(chunks):
        tries = 0
        success = False
        start = time.time()
        
        if debug:
            print(f"Loop {i} running through array of size {len(labels)}")

        while not success and tries < num_retries:
            result = _call_gpt(labels, model)
            alt_labels = _process_result(result)

            n_labels = len(labels)
            n_alts = len(alt_labels)
            
            if n_labels == n_alts:
                success = True
            else:
                tries += 1
                
                print(f'FAILED. {n_labels} labels but {n_alts} alts!')
                print(f'attempting retry # {tries}')
                
        if success:
            labels += alt_labels
        else:
            print(f"failed completely after {num_retries} retries.")
            return
        
        if debug:
            print(f"api call time = {time.time() - start}")

    return alt_labels

In [ ]:
# encode() also tokenizes if this is a sentence transformer but we don't use sentence-transformer
# # we only use transformer so doublecheck if this owrks the way we intent it to 

# but also, maybe we already use tiktoken to tokenize for gpt3/4.
# but waht about the output of gpt3/4? we still need to tokenize that appropriately 

def embed_labels(labels, embedding_model):
    labels = np.array(labels)
    embeddings = embedding_model.encode(labels)
    return embeddings

In [ ]:
# TODO: eliminate this thing since we should reuse previous embedding code 
def gpt_augmentation(ann_ids, maps, labels, embeddings, authors, embedding_model):
    augmented_annids, augmented_maps, augmented_labels, augmented_embeddings, augmented_authors = [], [], [], [], []
    alt_labels, _ = get_gpt_alt_labels(labels)
    print(len(alt_labels))
    print(len(labels))

    assert len(alt_labels) == len(labels)

    alt_label_embeddings = embed_labels(alt_labels, embedding_model)

    for i, (alt_label, alt_label_embedding) in enumerate(zip(alt_labels, alt_label_embeddings)):
        # append original ann_id twice
        augmented_annids.append(ann_ids[i])
        augmented_annids.append(ann_ids[i])

        augmented_authors.append(authors[i])
        augmented_authors.append(authors[i])

        # append original map twice
        augmented_maps.append(maps[i])
        augmented_maps.append(maps[i])

        # append the original label and the alt label
        augmented_labels.append(labels[i])
        augmented_labels.append(alt_label)

        # append the original embedding and the alt labels embedding
        augmented_embeddings.append(embeddings[i])
        augmented_embeddings.append(alt_label_embedding)



    return augmented_annids, augmented_maps, augmented_labels, augmented_embeddings, augmented_authors

# Tokenize Input

In [51]:
get_sent_word_embeddings(['I am a big foo bar'], model, tokenizer, None)

(array([[ 9.82737169e-02,  1.92254707e-02,  1.50002148e-02,
          2.88651371e-03, -5.50648011e-02, -3.68904248e-02,
          1.18874013e-01,  4.83325757e-02,  6.01447141e-03,
          4.21652161e-02, -3.52831781e-02, -8.36424306e-02,
         -3.50537300e-02, -1.95263885e-02,  2.79278532e-02,
          3.95715386e-02, -1.28606902e-02,  1.15014147e-02,
         -1.06786359e-02, -2.24973243e-02, -1.10274829e-01,
          6.65320083e-02,  1.88118499e-02,  1.76326197e-03,
         -1.19415283e-01, -1.26929749e-02,  4.31086402e-03,
          6.83599897e-03,  8.61172006e-02, -5.21420985e-02,
          1.39685320e-02,  7.56450966e-02,  4.12384309e-02,
         -1.83217507e-02, -8.99099484e-02, -2.86612986e-03,
          4.25359234e-02, -9.02812183e-03,  3.88544612e-02,
          4.52078208e-02,  2.26972681e-02, -9.90414154e-03,
          3.50649208e-02,  3.34182046e-02, -3.00184805e-02,
          1.63097437e-02, -2.29557529e-02,  4.05627266e-02,
          7.01322453e-04, -1.12383990e-0

In [ ]:
    # # load data, filter it, and generate the embeddings
    # ann_ids, maps, annotations, authors = get_db_data(cursor=mycursor)
    # print("Number of levels in db: ", len(ann_ids))
    # 
    # ann_ids, maps, annotations = sort_by_annid(ann_ids, maps, annotations)
    # sent_embeddings, word_embeddings = get_sent_word_embeddings(model, tokenizer, annotations, max_len=25)
    # data = {
    #         'ann_ids' : ann_ids,
    #         'images': maps,
    #         'labels': annotations,
    #         'embeddings': sent_embeddings,
    #         'word_embeddings': word_embeddings,
    #     }
    # np.save('datasets/Map Data/maps_noaug.npy', data, allow_pickle=True)

In [ ]:
# do gpt alt label augmentation
    ann_ids, maps, annotations, embeddings, authors = gpt_augmentation(ann_ids, maps, annotations, sent_embeddings, authors, model)
    print("Number of levels before filtering: ", len(ann_ids))

    ann_ids_exp = np.array(ann_ids)
    maps_exp = np.array(maps)
    annotations_exp = np.array(annotations)
    embeddings_exp = np.array(embeddings)


    export_data(ann_ids_exp, maps_exp, annotations_exp, embeddings_exp, 'datasets/maps_gpt4_aug.npy')